In [1]:
(defun makepipe (app &optional args)
  (let ((proc (sb-ext:run-program app args
                     :input :stream
                     :output :stream
                     :wait nil
                     :search t)))
  (when proc
    (make-two-way-stream (sb-ext:process-output proc)
                         (sb-ext:process-input proc)))))


MAKEPIPE


In [2]:
(defun readpipe (stream)
  (force-output stream)
  (with-output-to-string (out)
    (loop
       for c = (read-char-no-hang stream)
       while c do (write-char c out))))


READPIPE


In [3]:
(defun readpipe-complete (stream)
  "Read pipe until empty."
  (setq c (readpipe stream))
  (setq cc c)
  (loop until (equal c "") do 
    (progn
      (sleep 0.1)
      (setq c (readpipe stream))
      (setq cc (concatenate 'string cc c))
      (force-output stream)))
     cc)

READPIPE-COMPLETE


In [4]:
(defun writepipe (stream data)
    (format stream "~a~%" data)
    (finish-output stream))


(defun take-from-end (str n)
  "Return the last n characters or nil"
  (let ((l (length str)))
    (if (> l n) (subseq str (- l n)) nil)))


WRITEPIPE


In [5]:
(defparameter itups internal-time-units-per-second)

(defun delta-time (timestamp)
  "Delta time <== internal-real-time - timestamp in seconds"
  (/ (- (get-internal-real-time) timestamp) itups))



ITUPS


In [6]:
(defun readpipe-timeout (stream timeout)
  "Read pipe until empty or timeout in seconds."
  (setq c (readpipe stream))
  (setq cc c)
  (setq timestamp (get-internal-real-time))
  (loop until (or (equal c "") (> (delta-time timestamp) timeout)) do 
    (progn
      (sleep 0.1)
      (setq c (readpipe stream))
      (setq cc (concatenate 'string cc c))
      (force-output stream)))
     (if (equal c "") cc nil))


READPIPE-TIMEOUT


In [7]:
(defun redget ()
  (readpipe-timeout red *timeout*))

(defun redput (data)
  (writepipe red data))

(defun redeval (code)
  (writepipe red code)
  (readpipe-timeout red *timeout*))


REDGET


In [8]:
(setq red (makepipe "/usr/local/bin/redpsl"))

#<TWO-WAY-STREAM :INPUT-STREAM #<SB-SYS:FD-STREAM for "descriptor 8" {1005263203}> :OUTPUT-STREAM #<SB-SYS:FD-STREAM for "descriptor 7" {1005263043}> {10052634C3}>


In [9]:
(redget)

 The variable *TIMEOUT* is unbound.)


In [10]:
(defparameter *timeout* 5)

*TIMEOUT*


In [11]:
(redget)

 The function COMMON-LISP-USER::DELTA-TIME is undefined.)


In [12]:
(defun delta-time (timestamp)
  "Delta time <== internal-real-time - timestamp in seconds"
  (/ (- (get-internal-real-time) timestamp) itups))


DELTA-TIME


In [13]:
(redget)

In [14]:
(readpipe red)

In [15]:
(redput "x:=123456;")

 The function COMMON-LISP-USER::REDPUT is undefined.)


In [16]:
(defun redput (data)
  (writepipe red data))

REDPUT


In [17]:
(redput "x:=123456;")

NIL


In [18]:
(readpipe red)


x := 123456

2: 


In [33]:
(redput "df(sin(z),z);")

NIL


In [34]:
(readpipe red)



cos(z)

9: 


In [37]:
(redput "df(u**n,u,3);")

NIL


In [38]:
(readpipe red)


  n     2
 u *n*(n  - 3*n + 2)
---------------------
          3
         u

11: 
